In [1]:
import a2a
print('Success!')

from a2a.types import (
    AgentAuthentication,
    AgentCapabilities,
    AgentCard,
    AgentSkill,
)

Success!


In [ ]:
skill = AgentSkill(
        id='hello_world',
        name='Returns hello world',
        description='just returns hello world',
        tags=['hello world'],
        examples=['hi', 'hello world'],
    )

ac = AgentCard(
    name='Hello World Agent',
        description='Just a hello world agent',
        url='http://localhost:9999/', # Agent will run here
        version='1.0.0',
        defaultInputModes=['text'],
        defaultOutputModes=['text'],
        capabilities=AgentCapabilities(), # Basic capabilities
        skills=[skill], # Includes the skill defined above
        authentication=AgentAuthentication(schemes=['public']), # No auth needed
)
print(type(ac))

NameError: name 'skill' is not defined

In [1]:
import uuid
from datetime import datetime

def get_data():
    import requests

    res = requests.get("https://randomuser.me/api/")
    res = res.json()
    res = res['results'][0]

    return res
    # print(json.dumps(res, indent=3))

def format_data(res):
    data = {}
    location = res['location']
    data['id'] = uuid.uuid4().int
    data['first_name'] = res['name']['first']
    data['last_name'] = res['name']['last']
    data['gender'] = res['gender']
    data['address'] = f"{str(location['street']['number'])} {location['street']['name']}, " \
                      f"{location['city']}, {location['state']}, {location['country']}"
    data['post_code'] = location['postcode']
    data['email'] = res['email']
    data['username'] = res['login']['username']
    data['dob'] = res['dob']['date']
    data['registered_date'] = res['registered']['date']
    data['phone'] = res['phone']
    data['picture'] = res['picture']['medium']
    return data


In [6]:
import json
from kafka import KafkaProducer
import time
import logging

# producer = KafkaProducer(bootstrap_servers=['broker:29092'], max_block_ms=5000)
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], max_block_ms=5000)
curr_time = time.time()

try:
    res = get_data()
    res = format_data(res)
    print(res)

    producer.send('users_created', json.dumps(res).encode('utf-8'))
except Exception as e:
        logging.error(f'An error occured: {e}')

{'id': 145907818603750359228993983483136831673, 'first_name': 'Begüm', 'last_name': 'Akay', 'gender': 'female', 'address': '3850 Anafartalar Cd, Sivas, Hakkâri, Turkey', 'post_code': 35985, 'email': 'begum.akay@example.com', 'username': 'lazyladybug635', 'dob': '1949-08-21T16:32:40.310Z', 'registered_date': '2009-12-31T00:00:23.603Z', 'phone': '(560)-633-8195', 'picture': 'https://randomuser.me/api/portraits/med/women/11.jpg'}


In [7]:
from kafka import KafkaConsumer
import json
import time

consumer = KafkaConsumer(
    'users_created',
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

messages = consumer.poll(timeout_ms=1000)

In [9]:
print("Messages received:")
print(messages)
for topic_partition, messages in messages.items():
    for message in messages:
        print(f"Topic: {topic_partition.topic}, Partition: {topic_partition.partition}, Offset: {message.offset}, Key: {message.key}, Value: {message.value}")
        # print(f"Message: {message.value}")
        # print(f"Key: {message.key}")
        # print(f"Offset: {message.offset}")
        # print(f"Timestamp: {datetime.fromtimestamp(message.timestamp / 1000)}")

Messages received:
{TopicPartition(topic='users_created', partition=0): [ConsumerRecord(topic='users_created', partition=0, offset=1, timestamp=1747064770229, timestamp_type=0, key=None, value={'id': 190397088511188934512096305758284702733, 'first_name': 'Dirk', 'last_name': 'Klos', 'gender': 'male', 'address': '5380 Ahornweg, Calw, Hamburg, Germany', 'post_code': 15692, 'email': 'dirk.klos@example.com', 'username': 'ticklishbird312', 'dob': '1984-04-13T14:00:56.133Z', 'registered_date': '2011-04-03T00:05:06.641Z', 'phone': '0528-2737034', 'picture': 'https://randomuser.me/api/portraits/med/men/50.jpg'}, headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=417, serialized_header_size=-1), ConsumerRecord(topic='users_created', partition=0, offset=2, timestamp=1747074970826, timestamp_type=0, key=None, value={'id': 145907818603750359228993983483136831673, 'first_name': 'Begüm', 'last_name': 'Akay', 'gender': 'female', 'address': '3850 Anafartalar Cd, Sivas, Hakkâri, T